**Install Relevant Packages**

In [ ]:
!pip install numpy
!pip install --upgrade matplotlib
!pip install itertools
!pip install mne

ERROR: Could not find a version that satisfies the requirement itertools (from versions: none)
ERROR: No matching distribution found for itertools


**Import Relevant Packages**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import compress

import mne

**Load in Your Data**

In [ ]:
# This is an example motor dataset - put your folder path here!
fnirs_data_folder = mne.datasets.fnirs_motor.data_path()
fnirs_cw_amplitude_dir = fnirs_data_folder / "Participant-1"

# Load Data
raw_intensity = mne.io.read_raw_nirx(fnirs_cw_amplitude_dir, verbose=True)
raw_intensity.load_data()

Using default location ~/mne_data for fnirs_motor...
Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json
Loading /root/mne_data/MNE-fNIRS-motor-data/Participant-1


RuntimeError: ignored

**Add Annotations**

In [ ]:
raw_intensity.annotations.set_durations(5) # This sets the duration of stimulus (i.e. rock paper scissors time). Adjust as you need!
raw_intensity.annotations.rename(
    {"1.0": "Control", "2.0": "Tapping/Left", "3.0": "Tapping/Right"}) # Add other annotations here if you have multiple conditions
unwanted = np.nonzero(raw_intensity.annotations.description == "15.0")
raw_intensity.annotations.delete(unwanted)

**Selecting Appropriate Data Acquisition Channels**

In [ ]:
picks = mne.pick_types(raw_intensity.info, meg=False, fnirs=True)
dists = mne.preprocessing.nirs.source_detector_distances(
    raw_intensity.info, picks=picks
)
raw_intensity.pick(picks[dists > 0.01])
raw_intensity.plot(
    n_channels=len(raw_intensity.ch_names), duration=500, show_scrollbars=False
) # this plots the raw data so that we can view it

**Convert from Raw Intensity to Optical Density**

In [ ]:
raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)
raw_od.plot(n_channels=len(raw_od.ch_names), duration=500, show_scrollbars=False)

**Evaluate the quality of the data**

Here, we would like to remove channels which have a low scalp coupling index (which means they have a low heart rate signal, indicating poor signal to noise ratio). This code automatically removes those channels

In [ ]:
sci = mne.preprocessing.nirs.scalp_coupling_index(raw_od)
fig, ax = plt.subplots()
ax.hist(sci)
ax.set(xlabel="Scalp Coupling Index", ylabel="Count", xlim=[0, 1])
raw_od.info["bads"] = list(compress(raw_od.ch_names, sci < 0.5))


**Converting from Optical Density to Haemoglobin**

In [ ]:
raw_haemo = mne.preprocessing.nirs.beer_lambert_law(raw_od, ppf=0.1)
raw_haemo.plot(n_channels=len(raw_haemo.ch_names), duration=500, show_scrollbars=False)

**Removing Heart Rate from the signal**

This code will also plot the fourier transforms of the unflitered and filtered signals

In [ ]:
raw_haemo_unfiltered = raw_haemo.copy()
raw_haemo.filter(0.05, 0.7, h_trans_bandwidth=0.2, l_trans_bandwidth=0.02)
for when, _raw in dict(Before=raw_haemo_unfiltered, After=raw_haemo).items():
    fig = _raw.compute_psd().plot(average=True, picks="data", exclude="bads")
    fig.suptitle(f"{when} filtering", weight="bold", size="x-large")
    fig.subplots_adjust(top=0.88)

**Extract epochs (trials)**

In [ ]:
events, event_dict = mne.events_from_annotations(raw_haemo)

# The following lines plot the time location of each events. Use them to double check your annotations!
fig = mne.viz.plot_events(events, event_id=event_dict, sfreq=raw_haemo.info["sfreq"])
fig.subplots_adjust(right=0.7)  # make room for the legend

**Reject unreasonable epochs**

This code rejects epochs with physiologically unreasonable oxyhemoglobin levels

In [ ]:
reject_criteria = dict(hbo=80e-6)
tmin, tmax = -2, 15

epochs = mne.Epochs(
    raw_haemo,
    events,
    event_id=event_dict,
    tmin=tmin,
    tmax=tmax,
    reject=reject_criteria,
    reject_by_annotation=True,
    proj=True,
    baseline=(-2, 0),
    preload=True,
    detrend=None,
    verbose=True,
)
epochs.plot_drop_log()

**Topographically compare two conditions**

Use your annotations in epochs["My Condition Name"] to compare two conditions. Ask Ben if you need help on this part!

In [ ]:
%matplotlib inline
plt.ion()
topomap_args = dict(extrapolate="local")

# Below, replace "Tapping" and "Control" with your conditions
evoked_dict = {
    "Tapping/HbO": epochs["Tapping"].average(picks="hbo"),
    "Tapping/HbR": epochs["Tapping"].average(picks="hbr"),
    "Control/HbO": epochs["Control"].average(picks="hbo"),
    "Control/HbR": epochs["Control"].average(picks="hbr"),
}
for condition in evoked_dict:
    evoked_dict[condition].rename_channels(lambda x: x[:-4])

evoked_left = epochs["Tapping/Left"].average()
evoked_right = epochs["Tapping/Right"].average()

fig, axes = plt.subplots(
    nrows=2, ncols=4, figsize=(9, 5), gridspec_kw=dict(width_ratios=[1, 1, 1, 0.1])
)
vlim = (-8, 8)
ts = 9.0 # The time at which we would like to plot topographic distribution. You should change this


evoked_left.plot_topomap(ch_type="hbo", times=ts, axes=axes[0, 0], vlim=vlim, colorbar=False, **topomap_args)
evoked_left.plot_topomap(ch_type="hbr", times=ts, axes=axes[1, 0], vlim=vlim, colorbar=False, **topomap_args)
evoked_right.plot_topomap(ch_type="hbo", times=ts, axes=axes[0, 1], vlim=vlim, colorbar=False, **topomap_args)
evoked_right.plot_topomap(ch_type="hbr", times=ts, axes=axes[1, 1], vlim=vlim, colorbar=False, **topomap_args)

evoked_diff = mne.combine_evoked([evoked_left, evoked_right], weights=[1, -1])

evoked_diff.plot_topomap(ch_type="hbo", times=ts, axes=axes[0, 2:], vlim=vlim, colorbar=True, **topomap_args)
evoked_diff.plot_topomap(ch_type="hbr", times=ts, axes=axes[1, 2:], vlim=vlim, colorbar=True, **topomap_args)

for column, condition in enumerate(["Tapping Left", "Tapping Right", "Left-Right"]):
    for row, chroma in enumerate(["HbO", "HbR"]):
        axes[row, column].set_title("{}: {}".format(chroma, condition))
fig.tight_layout()
plt.show()

**Plot Channel activity and block average for one condition (or more if you want!)**

In [ ]:
epochs["Right"].plot_image(
    combine="mean",
    vmin=-30,
    vmax=30,
    ts_args=dict(ylim=dict(hbo=[-15, 15], hbr=[-15, 15])),
)

**Extract data to a csv file so you can manipulate it on your own**

In [ ]:
by_condition_data_this_subject = epochs.to_data_frame()
by_condition_data_this_subject.to_csv("This Subject Data.csv")